In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# installing linux related stuff for pyserini
# removing error "ImportError: No module named '_swigfaiss"
# Reference Link: https://github.com/facebookresearch/faiss/issues/821
!sudo apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 41 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 0s (593 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty

In [ ]:
# installing important packages for analyzing the code.
!pip install jsonlines
!pip install pyserini
!pip install faiss

In [4]:
# import statements
import pandas as pd
import jsonlines
import pyserini
from tqdm import tqdm
from xml.dom import minidom
import re
import nltk
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import STOPWORDS
import string
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Creating an index 

In [ ]:
# loading all the stop words frome english language.
stop_words = set(stopwords.words("english"))

In [ ]:
# query expansion corpus
qcorpus_list = list(jsonlines.open('/content/drive/MyDrive/Touche/touche-task2-passages-expanded-with-queries.jsonl'))

In [ ]:
# printing demo query
qcorpus_list[0]

{'chatNoirUrl': 'https://chatnoir.eu/cache?uuid=25e04d49-8df7-58c9-8fae-c5bd54a070ba&index=cw12&raw&plain',
 'contents': "Do Asian-Americans Face Bias in Admissions at Elite Colleges? - NYTimes.com Home Page Today's Paper Video Most Popular Times Topics Search All NYTimes.com Education World U.S. Politics Education Bay Area Chicago Texas N.Y. / Region Business Technology Science Health Sports Opinion Arts Style Travel Jobs Real Estate Autos February 8, 2012, 1:43 pm Do Asian-Americans Face Bias in Admissions at Elite Colleges? By DANIEL E. SLOTNIK 6:08 p.m. | Updated A statement from Princeton was added to the story. The Department of Education’s Office for Civil Rights is examining complaints thatHarvard and Princeton have discriminated against Asian-American undergraduate applicants, highlighting a concern of many Asian-American parents. The inquiry was first reported by Bloomberg News.<query> do asian americans face bias in admissions</query>",
 'id': 'clueweb12-0000tw-00-14115___1'

In [ ]:
!mkdir sample_collection_jsonl
!mkdir indexes
!mkdir indexes/sample_collection_jsonl

mkdir: cannot create directory ‘sample_collection_jsonl’: File exists
mkdir: cannot create directory ‘indexes’: File exists
mkdir: cannot create directory ‘indexes/sample_collection_jsonl’: File exists


In [ ]:
#creating an input file for indexing with preprocessing in the pyserini compatible format
output = jsonlines.open('sample_collection_jsonl/documents.jsonl', 'w')
for vals in tqdm(qcorpus_list):
    document = vals['contents']
    document = document.lower()
    # Remove punctuation and numbers
    document = re.sub(r"[^A-Za-z\s]+", " ", document)
    # Remove extra XML query tag
    document = document.replace("query"," ")
    # Remove extra white spaces
    document = re.sub(r"[\s]{2,}", " ", document)
    # Tokenize string
    word_tokens = word_tokenize(document)
    # Remove stop words
    word_tokens = [token for token in word_tokens if token not in stop_words]
    word_tokens = [token for token in word_tokens if len(token) > 1]
    # Combining word tokens for results.
    results = " ".join(word_tokens)
    document = results.strip()
    output.write({
        'id': vals['id'],
        'contents': document
    })

In [ ]:
!head -10 sample_collection_jsonl/documents.jsonl

{"id": "clueweb12-0000tw-00-14115___1", "contents": "asian americans face bias admissions elite colleges nytimes com home page today paper video popular times topics search nytimes com education world politics education bay area chicago texas region business technology science health sports opinion arts style travel jobs real estate autos february pm asian americans face bias admissions elite colleges daniel slotnik updated statement princeton added story department education office civil rights examining complaints thatharvard princeton discriminated asian american undergraduate applicants highlighting concern many asian american parents inquiry first reported bloomberg news asian americans face bias admissions"}
{"id": "clueweb12-0000tw-00-14115___10", "contents": "insisting upholding affirmative action college admissions ensure america continues extremely race sensitive society everything based race asian americans say take heart competition makes stronger cares ivy league mission n

In [ ]:
#index builder
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input sample_collection_jsonl \
  --index indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 4 \
  --storePositions --storeDocvectors --storeRaw

2022-04-28 20:42:39,446 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Setting log level to INFO
2022-04-28 20:42:39,448 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2022-04-28 20:42:39,448 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2022-04-28 20:42:39,449 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: sample_collection_jsonl
2022-04-28 20:42:39,449 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2022-04-28 20:42:39,450 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2022-04-28 20:42:39,450 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 4
2022-04-28 20:42:39,451 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Language: en
2022-04-28 20:42:39,451 INFO  [main] index.IndexCollection

In [ ]:
!ls /content/indexes/sample_collection_jsonl

_0.fdm	_0_Lucene80_0.dvd  _0_Lucene84_0.tim  _0.nvm  _0.tvx
_0.fdt	_0_Lucene80_0.dvm  _0_Lucene84_0.tip  _0.si   segments_1
_0.fdx	_0_Lucene84_0.doc  _0_Lucene84_0.tmd  _0.tvd  write.lock
_0.fnm	_0_Lucene84_0.pos  _0.nvd	      _0.tvm


In [ ]:
#saving the index files to the drive
!cp -r /content/indexes/sample_collection_jsonl /content/drive/MyDrive/Touche

# Intial Retrieval

In [ ]:
def parse_xml(path):
  answer_list = []
  xmldoc = minidom.parse(path)
  itemlist = xmldoc.getElementsByTagName('topics')
  topic_list = itemlist[0].getElementsByTagName('topic')
  for topic in topic_list:
    tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue,topic.getElementsByTagName('objects')[0].firstChild.nodeValue))
    answer_list.append(tuple_for_add)
  parsed=pd.DataFrame(answer_list, columns=["Number","Title","Objects"])
  return parsed

In [ ]:
topics=parse_xml("/content/drive/MyDrive/Touche/topics-task2.xml")
topics.head()

,Number,Title,Objects
0,2,"Which is better, a laptop or a desktop?","laptop, desktop"
1,3,"Which is better, Canon or Nikon?","Canon, Nikon"
2,8,What are the advantages and disadvantages of P...,"PHP, Python"
3,9,Why is Linux better than Windows?,"Linux, Windows"
4,12,Train or plane? Which is the better choice?,"Train, plane"


In [ ]:
#creating a dataframe for intial retrieval
from pyserini.search import SimpleSearcher
from pyserini.search.lucene import LuceneSearcher
searcher = LuceneSearcher('/content/indexes/sample_collection_jsonl')
h1=[]
for l,j in tqdm(zip(topics["Title"],topics["Number"])):
  hits = searcher.search(l, k=1500)
  for i in range(len(hits)):
    contents=str.split(hits[i].raw,'"contents" :',)[1]
    contents=contents.replace("}","")
    contents=contents.replace('"',"")
    contents=contents.replace('\n',"")
    h1.append({"title_id":j,"title":l,'doc_id': hits[i].docid,'score': hits[i].score,"content": contents})

  # Value of K determines the number of documents that can be returned.
  print('Number of document matches from the index: '+str(len(hits)))

1it [00:00,  1.73it/s]

Number of document matches from the index: 1500


3it [00:00,  3.47it/s]

Number of document matches from the index: 1500
Number of document matches from the index: 1500


5it [00:01,  4.52it/s]

...

Number of document matches from the index: 1500
Number of document matches from the index: 1500


48it [00:06,  8.29it/s]

Number of document matches from the index: 1500
Number of document matches from the index: 1500


50it [00:06,  7.27it/s]

Number of document matches from the index: 1500


In [ ]:
df=pd.DataFrame(h1)
df.head()

,title_id,title,doc_id,score,content
0,2,"Which is better, a laptop or a desktop?",clueweb12-0608wb-66-03868___1,10.0240,desktop laptop tech homeforumscontact hijackt...
1,2,"Which is better, a laptop or a desktop?",clueweb12-0001wb-72-30848___8,9.9780,desktop replacements usually larger heavier s...
2,2,"Which is better, a laptop or a desktop?",clueweb12-0006wb-17-30732___8,9.8379,gateway nv affordable offers memory ram bette...
3,2,"Which is better, a laptop or a desktop?",clueweb12-0907wb-60-03588___2,9.7714,turnaround pricing trends past months whereas...
4,2,"Which is better, a laptop or a desktop?",clueweb12-1804wb-90-15876___2,9.7203,therefore hard drives laptop usually relative...


In [ ]:
#saving the 1500 initial retrievals for each query
df.to_csv("/content/drive/MyDrive/Touche/Final_Relevant_BM25.csv", index=False)

#Psuedo Relevance Feedback Initial Retreival

In [5]:
topics=pd.read_csv("/content/drive/MyDrive/Touche/topics-task2-psuedo_relevance.csv")
topics.head()

,Number,Title,Objects,New,Frequency
0,2,"Which is better, a laptop or a desktop?","laptop, desktop","Which is better, a laptop or a desktop? computer",39
1,3,"Which is better, Canon or Nikon?","Canon, Nikon","Which is better, Canon or Nikon? vs",42
2,8,What are the advantages and disadvantages of P...,"PHP, Python",What are the advantages and disadvantages of P...,45
3,9,Why is Linux better than Windows?,"Linux, Windows",Why is Linux better than Windows? software,12
4,12,Train or plane? Which is the better choice?,"Train, plane",Train or plane? Which is the better choice? tr...,25


In [ ]:
#creating a dataframe for intial retrieval
from pyserini.search import SimpleSearcher
from pyserini.search.lucene import LuceneSearcher
searcher = LuceneSearcher('/content/drive/MyDrive/Touche/sample_collection_jsonl')
h1=[]
for l,j,m in tqdm(zip(topics["New"],topics["Number"],topics["Title"])):
  hits = searcher.search(l, k=1500)
  for i in range(len(hits)):
    contents=str.split(hits[i].raw,'"contents" :',)[1]
    contents=contents.replace("}","")
    contents=contents.replace('"',"")
    contents=contents.replace('\n',"")
    h1.append({"title_id":j,"title":m,"New_title":l,'doc_id': hits[i].docid,'score': hits[i].score,"content": contents})

  # Value of K determines the number of documents that can be returned.
  print('Number of document matches from the index: '+str(len(hits)))

In [ ]:
df=pd.DataFrame(h1)
df.head()

In [9]:
df.to_csv("/content/drive/MyDrive/Touche/Final_Relevant_BM25_psuedo_relevance.csv", index=False)